## Dataset - Data Cleaning

Fichiers a traiter :
- dataset/marmiton/data/recettes-plats.csv
- dataset/marmiton/data/recettes-desserts.csv
- dataset/marmiton/data/recettes-boissons.csv
- dataset/marmiton/data/recettes-entrees.csv

Methodologie :
L'ojectif est de nettoyer et de preparer les donnees pour la modelisation. Il convient en premiere de fusionner les 4 datasets.
Certaines transformation peuvent etre realiser sur les datasets avant la fusion, comme l'ajout du type de recette.
Une fois les 4 datasets reunis, il sera plus facile de supprimer les doublons, les individus Null, convertir les listes, les durees en timestamp, etc.

### Chargement des librairies

In [224]:
import pandas as pd
import ast
from datetime import datetime, timedelta
import re
import numpy as np

## 1. Charger les datasets et attribuer un type aux recettes

In [225]:
DATASET_PLATS = "dataset/marmiton/data/recettes-plats.csv"
DATASET_ENTREES = "dataset/marmiton/data/recettes-entrees.csv"
DATASET_DESSERTS = "dataset/marmiton/data/recettes-desserts.csv"
DATASET_BOISSONS = "dataset/marmiton/data/recettes-boissons.csv"


df_plats = pd.read_csv(DATASET_PLATS, sep=";", encoding="utf-8")    
df_entrees = pd.read_csv(DATASET_ENTREES, sep=";", encoding="utf-8")
df_desserts = pd.read_csv(DATASET_DESSERTS, sep=";", encoding="utf-8")
df_boissons = pd.read_csv(DATASET_BOISSONS, sep=";", encoding="utf-8")

In [226]:
df_plats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17070 entries, 0 to 17069
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Titre                17029 non-null  object 
 1   Temps Total          16963 non-null  object 
 2   Temps Préparation    17029 non-null  object 
 3   Temps Cuisson        17029 non-null  object 
 4   Nombre de personnes  17022 non-null  float64
 5   Difficulté           17029 non-null  object 
 6   Coût                 17029 non-null  object 
 7   Ingrédients          17070 non-null  object 
 8   Étapes               17070 non-null  object 
 9   Ustensiles           17070 non-null  object 
 10  Liens_recette        17070 non-null  object 
dtypes: float64(1), object(10)
memory usage: 1.4+ MB


In [227]:
df_entrees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Titre                5549 non-null   object 
 1   Temps Total          5531 non-null   object 
 2   Temps Préparation    5549 non-null   object 
 3   Temps Cuisson        5549 non-null   object 
 4   Nombre de personnes  5545 non-null   float64
 5   Difficulté           5549 non-null   object 
 6   Coût                 5549 non-null   object 
 7   Ingrédients          5570 non-null   object 
 8   Étapes               5570 non-null   object 
 9   Ustensiles           5570 non-null   object 
 10  Liens_recette        5570 non-null   object 
dtypes: float64(1), object(10)
memory usage: 478.8+ KB


In [228]:
df_desserts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12841 entries, 0 to 12840
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Titre                11281 non-null  object
 1   Temps Total          11234 non-null  object
 2   Temps Préparation    11281 non-null  object
 3   Temps Cuisson        11281 non-null  object
 4   Nombre de personnes  11268 non-null  object
 5   Difficulté           11281 non-null  object
 6   Coût                 11281 non-null  object
 7   Ingrédients          12841 non-null  object
 8   Étapes               12841 non-null  object
 9   Ustensiles           12841 non-null  object
 10  Liens_recette        12841 non-null  object
dtypes: object(11)
memory usage: 1.1+ MB


In [229]:
df_boissons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Titre                507 non-null    object 
 1   Temps Total          500 non-null    object 
 2   Temps Préparation    507 non-null    object 
 3   Temps Cuisson        507 non-null    object 
 4   Nombre de personnes  507 non-null    float64
 5   Difficulté           507 non-null    object 
 6   Coût                 507 non-null    object 
 7   Ingrédients          511 non-null    object 
 8   Étapes               511 non-null    object 
 9   Ustensiles           511 non-null    object 
 10  Liens_recette        511 non-null    object 
dtypes: float64(1), object(10)
memory usage: 44.0+ KB


### Ajouter le type des plats

In [230]:
df_plats['Type'] = 'plat'
df_entrees['Type'] = 'entree'
df_desserts['Type'] = 'dessert'
df_boissons['Type'] = 'boisson'

### Fusionner les datasets

In [231]:
# Fusionner les quatres dataset
df = pd.concat([df_plats, df_entrees, df_desserts, df_boissons], ignore_index=True)

In [232]:
df.head()

,Titre,Temps Total,Temps Préparation,Temps Cuisson,Nombre de personnes,Difficulté,Coût,Ingrédients,Étapes,Ustensiles,Liens_recette,Type
0,Cuisse de dinde façon couscous de chez Karpeth,1h50,25 min,1h25,4.0,moyenne,moyen,"['2 courgettes , diamètre 4 ou 5 cm', '3 carot...",['1 casserole pour préparer la semoule (ou 1 b...,"['1 couteau', '1 Couvercle', '1 sauteuse', '1 ...",https://www.marmiton.org/recettes/recette_cuis...,plat
1,Blanquette de veau facile au Cookeo,1h10,25 min,45 min,4.0,très facile,moyen,"['2 carottes', '1 oignon jaune', '1 petit pot ...","[""Coupez la viande en morceaux de taille moyen...","['1 Cookeo', '1 Cuillère en bois', '1 bol', '1...",https://www.marmiton.org/recettes/recette_blan...,plat
2,Terrine de foie gras au Sauternes,1h40,40 min,1 h,10.0,facile,assez cher,"['3 pincées de quatre-épices', '15 g de sel', ...","['A préparer 4 ou 5 jours avant.', ""Faire trem...","['1 Cuillère en bois', '1 saladier', '1 Four',...",https://www.marmiton.org/recettes/recette_terr...,plat
3,Couscous Royal,3h30,30 min,3 h,2.0,facile,bon marché,"['1 barquette d’émincé 100% végétal ACCRO', '4...",['Préparation de la semoule : placez la semoul...,"['1 plat', '1 casseroles', '1 couteau']",https://www.marmiton.org/recettes/recette_cous...,plat
4,"Quiche jambon, fromage, tomate, olives",40 min,15 min,25 min,6.0,facile,bon marché,"[""4 tranches de jambon cuit à l'Étouffée LE BO...",['Préchauffez le four à 200°C en chaleur tourn...,"['1 moule à tarte', '1 fourchette', '1 saladier']",https://www.marmiton.org/recettes/recette_quic...,plat


## 2. Data Cleaning

### Supprimer les doublons

In [233]:
# Verifier les doublons a partir du titre de la recette
df.duplicated(subset=['Titre']).sum()

np.int64(1706)

In [234]:
# Supprimer les doublons a partir du titre de la recette
df = df.drop_duplicates(subset=['Titre'], keep='first')

### Supprimer les valeurs NaN

In [235]:
df.isna().sum()

Titre                    1
Temps Total            139
Temps Préparation        1
Temps Cuisson            1
Nombre de personnes     23
Difficulté               1
Coût                     1
Ingrédients              0
Étapes                   0
Ustensiles               0
Liens_recette            0
Type                     0
dtype: int64

In [236]:
# Supprimer les individus sans titre
df = df.dropna(subset=['Titre'])

In [237]:
# Supprimer les individus sans nombre de personnes
df = df.dropna(subset=['Nombre de personnes'])

### Convertir les durees en minutes

In [238]:
def convertir_duree_en_minutes(duree):
    if pd.isnull(duree):
        return 0

    # Expressions régulières
    patterns = {
        'j': r'(\d+)\s*j',
        'h': r'(\d+)\s*h',
        'min': r'(\d+)\s*min',
        'sec': r'(\d+)\s*sec',
        'h_min': r'(\d+)h(\d+)?'
    }

    jours = int(re.search(patterns['j'], duree).group(1)) * 24 * 60 if re.search(patterns['j'], duree) else 0
    heures = int(re.search(patterns['h'], duree).group(1)) * 60 if re.search(patterns['h'], duree) else 0
    minutes = int(re.search(patterns['min'], duree).group(1)) if re.search(patterns['min'], duree) else 0
    secondes = int(re.search(patterns['sec'], duree).group(1)) if re.search(patterns['sec'], duree) else 0

    # Format compact sans espace "1h30"
    if re.search(patterns['h_min'], duree):
        match = re.search(patterns['h_min'], duree)
        heures = int(match.group(1)) * 60
        minutes = int(match.group(2)) if match.group(2) else 0

    total_minutes = jours + heures + minutes + secondes / 60
    return round(total_minutes)


In [239]:
# Convertir la colonne 'Temps Total' en minutes
df['temps_total_minutes'] = df['Temps Total'].apply(convertir_duree_en_minutes)
df['temps_preparation_minutes'] = df['Temps Préparation'].apply(convertir_duree_en_minutes)
df['temps_cuisson_minutes'] = df['Temps Cuisson'].apply(convertir_duree_en_minutes)

In [240]:
# Supprimer les anciennes colonnes
df.drop(columns=['Temps Total', 'Temps Préparation', 'Temps Cuisson'], inplace=True)

In [241]:
# Renommer les colonnes
df.rename(columns={'Titre': 'titre',
                   'Nombre de personnes': 'nombre_personnes',
                   'Ingrédients': 'ingredients',
                   'Étapes': 'etapes',
                   'Type': 'type',
                   'Ustensiles': 'ustensiles',
                   'Difficulté': 'difficulte',
                   'Coût': 'cout',
                   'Liens_recette': 'lien_recette'}, inplace=True)

In [242]:
# Modifier l'ordre des colonnes
# Réorganiser l'ordre des colonnes
nouvel_ordre = ['titre', 'type', 'temps_total_minutes', 'temps_preparation_minutes', 'temps_cuisson_minutes', 
                'nombre_personnes', 'difficulte', 'cout', 'ingredients', 
                'etapes', 'ustensiles', 'lien_recette']
df = df[nouvel_ordre]

### Imputation de la colonne 'temps_total_minutes' en faisant l'addition de 'temps_preparation_minutes' et 'temps_cuisson_minutes'

In [243]:
# Remplacer les valeurs egales a 0 dans la colonne 'temps_total_minutes' par la somme de 'temps_preparation_minutes' et 'temps_cuisson_minutes'
df.loc[df['temps_total_minutes'] == 0, 'temps_total_minutes'] = df['temps_preparation_minutes'] + df['temps_cuisson_minutes']

In [244]:
# Nombre de recettes avec un temps total egal a 0
df[df['temps_total_minutes'] == 0].shape[0]

134

In [245]:
# Supprimer les colonnes ou la duree est egal a 0
df = df[df['temps_total_minutes'] != 0]

### Convertir le nombre de personne en type entier

In [246]:
# Convertir la colonne 'Nombre de personnes' en entier
df['nombre_personnes'] = pd.to_numeric(df['nombre_personnes'], errors='coerce')  # convertit en float, les erreurs deviennent NaN
df['nombre_personnes'] = df['nombre_personnes'].fillna(0).astype(int)

In [247]:
# Verifier les recettes qui ont un nombre de personnes egale a 0 et supprimer les individus concerner
df[df['nombre_personnes'] == 0].shape[0]
df = df[df['nombre_personnes'] != 0]

In [248]:
df.head()

,titre,type,temps_total_minutes,temps_preparation_minutes,temps_cuisson_minutes,nombre_personnes,difficulte,cout,ingredients,etapes,ustensiles,lien_recette
0,Cuisse de dinde façon couscous de chez Karpeth,plat,110,25,85,4,moyenne,moyen,"['2 courgettes , diamètre 4 ou 5 cm', '3 carot...",['1 casserole pour préparer la semoule (ou 1 b...,"['1 couteau', '1 Couvercle', '1 sauteuse', '1 ...",https://www.marmiton.org/recettes/recette_cuis...
1,Blanquette de veau facile au Cookeo,plat,70,25,45,4,très facile,moyen,"['2 carottes', '1 oignon jaune', '1 petit pot ...","[""Coupez la viande en morceaux de taille moyen...","['1 Cookeo', '1 Cuillère en bois', '1 bol', '1...",https://www.marmiton.org/recettes/recette_blan...
2,Terrine de foie gras au Sauternes,plat,100,40,60,10,facile,assez cher,"['3 pincées de quatre-épices', '15 g de sel', ...","['A préparer 4 ou 5 jours avant.', ""Faire trem...","['1 Cuillère en bois', '1 saladier', '1 Four',...",https://www.marmiton.org/recettes/recette_terr...
3,Couscous Royal,plat,210,30,180,2,facile,bon marché,"['1 barquette d’émincé 100% végétal ACCRO', '4...",['Préparation de la semoule : placez la semoul...,"['1 plat', '1 casseroles', '1 couteau']",https://www.marmiton.org/recettes/recette_cous...
4,"Quiche jambon, fromage, tomate, olives",plat,40,15,25,6,facile,bon marché,"[""4 tranches de jambon cuit à l'Étouffée LE BO...",['Préchauffez le four à 200°C en chaleur tourn...,"['1 moule à tarte', '1 fourchette', '1 saladier']",https://www.marmiton.org/recettes/recette_quic...


### Convertir les ingredients, etapes et ustensiles en liste

In [249]:
# Convertir les ingredients en liste
df['ingredients'] = df['ingredients'].apply(ast.literal_eval)

In [250]:
# Convertir les etapes en liste
df['etapes'] = df['etapes'].apply(ast.literal_eval)

In [251]:
# Convertir les ustensiles en liste
df['ustensiles'] = df['ustensiles'].apply(ast.literal_eval)

In [252]:
df.head()

,titre,type,temps_total_minutes,temps_preparation_minutes,temps_cuisson_minutes,nombre_personnes,difficulte,cout,ingredients,etapes,ustensiles,lien_recette
0,Cuisse de dinde façon couscous de chez Karpeth,plat,110,25,85,4,moyenne,moyen,"[2 courgettes , diamètre 4 ou 5 cm, 3 carottes...",[1 casserole pour préparer la semoule (ou 1 bo...,"[1 couteau, 1 Couvercle, 1 sauteuse, 1 cassero...",https://www.marmiton.org/recettes/recette_cuis...
1,Blanquette de veau facile au Cookeo,plat,70,25,45,4,très facile,moyen,"[2 carottes, 1 oignon jaune, 1 petit pot de cr...",[Coupez la viande en morceaux de taille moyenn...,"[1 Cookeo, 1 Cuillère en bois, 1 bol, 1 Mijote...",https://www.marmiton.org/recettes/recette_blan...
2,Terrine de foie gras au Sauternes,plat,100,40,60,10,facile,assez cher,"[3 pincées de quatre-épices, 15 g de sel, 1 pi...","[A préparer 4 ou 5 jours avant., Faire tremper...","[1 Cuillère en bois, 1 saladier, 1 Four, 1 cou...",https://www.marmiton.org/recettes/recette_terr...
3,Couscous Royal,plat,210,30,180,2,facile,bon marché,"[1 barquette d’émincé 100% végétal ACCRO, 4 me...",[Préparation de la semoule : placez la semoule...,"[1 plat, 1 casseroles, 1 couteau]",https://www.marmiton.org/recettes/recette_cous...
4,"Quiche jambon, fromage, tomate, olives",plat,40,15,25,6,facile,bon marché,[4 tranches de jambon cuit à l'Étouffée LE BON...,[Préchauffez le four à 200°C en chaleur tourna...,"[1 moule à tarte, 1 fourchette, 1 saladier]",https://www.marmiton.org/recettes/recette_quic...


### Enregistrer le dataset

In [253]:
df.to_csv("dataset/marmiton/data/dataset-recettes-v1.csv", sep=";", encoding="utf-8", index=False)